Code for the analysis and visualizations for the paper "A similarity search approach to patent classification" by Reza Rezazadegan and Zahra Bagheri

www.github.com/rezareza007
www.dreamintelligent.com



In [13]:
import pandas as pd

import gc
import os.path
from tqdm import tqdm

import dask.dataframe as dd
import nltk
#nltk.download('wordnet')

fulltext_path="data/fulltext.tsv"
preprocessed_path="data/fulltext_preprocessed.tsv"



In [3]:
# Text preprocessing function

from nltk.stem  import PorterStemmer 
from nltk.stem import WordNetLemmatizer

from gensim.parsing.preprocessing import STOPWORDS
from  gensim.utils import simple_preprocess


ps = PorterStemmer()


def text_preprocess(text):

   
    
    if text=="":
        return []
    
    
    result = []
    for token in simple_preprocess(text):
        
        
        if token not in STOPWORDS and len(token) > 3:
            result.append(ps.stem(WordNetLemmatizer().lemmatize(token, pos='v') ))

    return result

In [5]:
# The path for brieftext and claims data files

rpath=r"../databases/PatentsView/"

def path(year, type):
    if type=='brief':
        pathh=rpath+'BriefText/'+'brf_sum_text_'+str(year)+'.tsv.zip'
        if year>=2004:
            pathh=pathh[:-4]
        return pathh
    else:
        return rpath+'Claims/'+'claims_'+str(year)+'.tsv.zip' 


In [6]:
from pandarallel import pandarallel
pandarallel.initialize()


for year in tqdm(range(1976, 2022)):
    

    print(year)


    textcol='text' if year not in [2004, 2005, 2013, 2014, 2011, 2017, 2018, 2019] else 'summary_text'


    # patent Brief texts
    df1=pd.read_csv(path(year, 'brief'),sep='\t', usecols=['patent_id', textcol]) #,chunksize=1000)
    df1=df1.rename(columns={textcol:'brief_text'})
    df1=df1.dropna()

    df1.brief_text=df1.brief_text.parallel_apply(lambda x: str(x).replace( '\n',' '  ).replace('\t', ' ')).replace(r"\r", "").replace("\"", "" ) 


    df1=df1[df1.patent_id.astype(str).str.isnumeric()]

    #patent claims
    df3=pd.read_csv(path(year, 'claim'),sep='\t',usecols=['patent_id','text']) #chunksize=5000,
    df3=df3.dropna()

    df3=df3.rename(columns={'text':'claim_text'})
    df3=df3[df3.patent_id.astype(str).str.isnumeric()]

    df3.claim_text=df3.claim_text.parallel_apply(lambda x: str(x).replace( '\n',' '  ).replace('\t', ' ').replace(r"\r", "").replace("\"", "" ) )


    df_g=df3.groupby('patent_id', sort=False)['claim_text'].apply(' '.join)
    df_claim=pd.DataFrame(df_g)


    del df3
    del df_g
    gc.collect()

    #merging the two
    df_merge=df1.merge(df_claim,on='patent_id', how='left')

    # Here fulltext means merger of brieftext and claims 

    df_merge['full_text']=df_merge.parallel_apply(lambda X: str(X.brief_text)+" "+str(X.claim_text), axis=1)
    df_merge=df_merge[['patent_id','full_text']]

    df_merge=df_merge.rename(columns={'patent_id': 'patent_number'})

    
    print("Writing merged texts to disk")
    df_merge.to_csv(fulltext_path,index=False, header=not os.path.exists(fulltext_path), mode='a', sep='\t')

    #Applying preprocess function
    df_merge['text_preprocessed']=df_merge.full_text.apply(lambda x: " ".join(text_preprocess))
    df_merge[['patent_number', 'text_preprocessed']].to_csv(preprocessed_path,index=False, header=not os.path.exists(preprocessed_path), mode='a', sep='\t')



    #it's not needed anymore
    del df_merge
    gc.collect()





INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


  0%|          | 0/46 [00:00<?, ?it/s]

1976


  2%|▏         | 1/46 [18:22<13:46:56, 1102.60s/it]

1977


...

C:\Users\Asus\AppData\Local\Temp\ipykernel_15216\887955281.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(path(year, 'brief'),sep='\t', usecols=['patent_id', textcol]) #,chunksize=1000)
C:\Users\Asus\AppData\Local\Temp\ipykernel_15216\887955281.py:37: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df3=pd.read_csv(path(year, 'claim'),sep='\t',usecols=['patent_id','text']) #chunksize=5000,
 96%|█████████▌| 44/46 [33:19:54<3:08:29, 5654.90s/it]

2020


C:\Users\Asus\AppData\Local\Temp\ipykernel_15216\887955281.py:37: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df3=pd.read_csv(path(year, 'claim'),sep='\t',usecols=['patent_id','text']) #chunksize=5000,
 98%|█████████▊| 45/46 [35:19:08<1:41:44, 6104.68s/it]

2021


C:\Users\Asus\AppData\Local\Temp\ipykernel_15216\887955281.py:37: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df3=pd.read_csv(path(year, 'claim'),sep='\t',usecols=['patent_id','text']) #chunksize=5000,
100%|██████████| 46/46 [37:20:36<00:00, 2922.53s/it]  


# word2vec model

In [6]:
import pickle
f=open("data/dictionary_full", "rb")
dictionary=pickle.load(f)
f.close()

In [1]:

class SentenceIterator: 
    def __init__(self, filepath): 
        self.filepath = filepath 

    def __iter__(self): 
        for line in open(self.filepath): 
            #print(line.split()[1:])
            yield line.split() [1:] #dictionary.doc2bow(line.split()[1:])   

sentences = SentenceIterator("data/fulltext_preprocessed.tsv") 





In [3]:
from gensim.models import Word2Vec
word2vec_model=Word2Vec(sentences, workers=20)

In [ ]:
word2vec_model.save('data/word2vec_full.model')